In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor

%matplotlib inline

In [ ]:
df_test = pd.read_csv('../input/test.csv', sep=',')
df_train = pd.read_csv('../input/train.csv', sep=',')
df_store=pd.read_csv('../input/store.csv',sep=',')

Посмотрим все ли значения заполнены

In [ ]:
df_test.isnull().any()

In [ ]:
df_train.isnull().any()

In [ ]:
df_store.isnull().any()

Заполним значения open значениями 1 в тестовой выборке

In [ ]:
df_test.fillna(1,inplace=True)

Посмотрим на данные тестовой и тренировочной выборок

In [ ]:
df_train.head()

In [ ]:
df_test.head()

Строка даты может быть не очень информативна для алгоритмов машинного обучения. Преобразуем дату в год и месяц

In [ ]:
df_train["Year"]=df_train["Date"].apply(lambda x: int(x[:4]))
df_test["Year"]=df_train["Date"].apply(lambda x: int(x[:4]))

In [ ]:
df_train["Month"]=df_train["Date"].apply(lambda x: int(x[5:7]))
df_test["Month"]=df_train["Date"].apply(lambda x: int(x[5:7]))

In [ ]:
df_train.head()

Поле StateHoliday является строкой

In [ ]:
df_train.StateHoliday.unique()

Посмотрим, как прибыль зависит от значения StateHoliday

In [ ]:
df_train.groupby("StateHoliday")["Sales"].mean().plot(kind="bar")

Можно видеть, что 0 и '0' имеют одно и тоже значение (праздника нет), а в случае 'a','b','c' есть праздник, из-за чего видимо магазины работают не полный день

In [ ]:
df_train["StateHoliday2"] = df_train["StateHoliday"].replace(0,"0").map({"0": 0, "a": 1, "b": 1, "c": 1})
df_test["StateHoliday2"]= df_test["StateHoliday"].replace(0,"0").map({"0": 0, "a": 1, "b": 1, "c": 1})

In [ ]:
df_train["StateHoliday2"].unique()

In [ ]:
df_train["SchoolHoliday"].unique()

In [ ]:
df_test.head()

Уберём лишнюю информацию

In [ ]:
df_train.drop(["Date","StateHoliday"],axis=1,inplace=True)
df_test.drop(["Date","StateHoliday"],axis=1,inplace=True)
df_train.head()

In [ ]:
df_test.head()

Попробуем предсказать

In [ ]:
train_stores = dict(list(df_train.groupby('Store')))
test_stores = dict(list(df_test.groupby('Store')))
submission = pd.Series()

In [ ]:
for i in test_stores:
    store = train_stores[i]
    
    X_train = store.drop(["Store","Sales", "Customers"],axis=1).values
    Y_train = store["Sales"].values
    
    store_ids = test_stores[i]["Id"]
    X_test  = test_stores[i].drop(["Id","Store"],axis=1).values
    
    model = RandomForestRegressor(n_estimators=25)
    model.n_jobs = 8
    model.fit(X_train,Y_train)
    Y_test=model.predict(X_test)
    
    submission = submission.append(pd.Series(Y_test, index=store_ids))

submission = pd.DataFrame({ "Id": submission.index, "Sales": submission.values})
submission.to_csv('submission.csv', index=False)